In [2]:
from collections import defaultdict
import pandas as pd
import sys
import pickle

In [80]:
def loadMovieData():
    userCol = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
    users = pd.read_csv('u.user', sep='|', names=userCol, engine='python')
    ratingCol = ['UserID', 'MovieID', 'Rating', 'Timestamp']
    ratings = pd.read_csv('u.data', sep='\t', names=ratingCol, engine='python')
    movieCol = ['MovieID', 'Title', 'Date']
    movies = pd.read_csv('u.item', sep='|', names=movieCol, engine='python',usecols = [0,1,2], index_col=0)
    return users, ratings, movies

# 加载数据集

In [81]:
usersdf, ratingsdf, moviesdf = loadMovieData()

# 只取评分高于3的数据项，视为用户喜欢的电影

In [4]:
ratingsdf["Favorable"] = ratingsdf["Rating"] > 3
ratingsdf = ratingsdf[ratingsdf['Favorable']].drop('Favorable', axis=1)

# 创建字典，key为用户ID，value为用户喜欢的所有电影的frozenset

In [5]:
usersFavorableMoviesDict = dict((k, frozenset(v.values))for k, v in ratingsdf.groupby("UserID")["MovieID"])

# 在所有电影数目为k的集合中找出满足最小支持度的频繁集，并创建字典，key为k部电影的fronzeset，value为对应的支持度

In [6]:
def findFreqSet(usersFavorableMoviesDict, kMovieIter, minSupport):
    countDict = defaultdict(int)
    for user, favorMovieSet in usersFavorableMoviesDict.items():
        for kMovieSet in kMovieIter:
            if favorMovieSet.issuperset(kMovieSet):
                countDict[kMovieSet] += 1
    return dict([(kMovieSet, count) for kMovieSet, count in countDict.items() if count >= minSupport])

# 利用k部电影的频繁集与单部电影的频繁集生成k+1部电影的集合，调用上个函数来找出其中的频繁集

In [7]:
def getKPlusFreqSet(kMovieList, singleMovieList, usersFavorMoviesDict, minSupport=50):
    kPlusMovie = dict()
    k = len(kMovieList[0])
    for kMovieSet in kMovieList:
        for singleMovie in singleMovieList:
            temp = singleMovie | kMovieSet
            if len(temp) == k:
                continue
            kPlusMovie[singleMovie | kMovieSet] = 1
    return findFreqSet(usersFavorMoviesDict, kPlusMovie.keys(), minSupport)

# 根据最小支持度先生成单部电影的频繁，然后迭代生成k部电影的频繁集，直至频繁集为空，最后用pickle模块保存至硬盘

In [8]:
def findAllFreqSet(ratingsdf,minSupport = 100):
    usersFavorMoviesDict = dict((k, frozenset(v.values))for k, v in ratingsdf.groupby("UserID")["MovieID"])
    movieFavorNums = ratingsdf.groupby('MovieID')['UserID'].count()
    singleMovieID = movieFavorNums[movieFavorNums >= minSupport].index
    singleMovieList = [frozenset([movieID]) for movieID in singleMovieID]
    singleMovieDict = movieFavorNums[movieFavorNums >= minSupport].to_dict()
    with open('1movieFreqDict','wb') as f:
        pickle.dump(singleMovieDict, f)
    kMovieList = singleMovieList
    for k in range(2, 100):
        print(k)
        sys.stdout.flush()
        kMovieDict = getKPlusFreqSet(kMovieList, singleMovieList, usersFavorMoviesDict, minSupport)
        if not kMovieDict:
            break
        kMovieList = list(kMovieDict.keys())
        with open('{!s}movieFreqDict'.format(k), 'wb') as f:
            pickle.dump(kMovieDict, f)
        del kMovieDict

# 以最小支持度为100，找出所有频繁集

In [ ]:
findAllFreqSet(ratingsdf, minSupport=100)

# 读出所有频繁集，放入字典

In [3]:
import re,os
freqSetDict = dict()
fileList = [file for file in os.listdir() if "movieFreqDict" in file]
for file in fileList:
    num = int(re.match('\d+', file).group())
    with open(file, 'rb') as f:
        freqSetDict[num] = pickle.load(f)

In [22]:
from pylab import *
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
%matplotlib
import seaborn as sns
sns.set_context("talk")

Using matplotlib backend: Qt5Agg


In [23]:
kMovieCount = dict((k, len(v)) for k, v in freqSetDict.items())
ax = sns.barplot(x=list(kMovieCount.keys()),y=list(kMovieCount.values()))
ax.set_title('频繁集数随电影数k的变化')
ax.set_xlabel('电影数k')
ax.set_ylabel('频繁集数')

![频繁集数随电影数k的变化](freqSetCount.png "Optional title")

# 取出所有k=5的频繁集计算其中每一部电影相对于k=4的置信度

In [124]:
confidenceDict = defaultdict(dict)
confidenceDictEachMovie = defaultdict(list)
for movieFreqSet in freqSetDict[5]:
    count = freqSetDict[5][movieFreqSet]
    for movie in movieFreqSet:
        confidence = count/freqSetDict[4][movieFreqSet - frozenset([movie])]
        confidenceDict[movieFreqSet - frozenset([movie])][movie]=confidence
        confidenceDictEachMovie[movie].append(confidence)

# 该函数可以将电影ID或者电影ID集合转换为对应电影标题的字符串

In [115]:
def getTitleByMovieID(movie):
    if isinstance(movie, frozenset):
        return (',').join([moviesdf.ix[movieID].Title for movieID in movie])
    else:
        try:
            return moviesdf.ix[int(movie)].Title
        except:
            raise TypeError("the input must be the movieID number or the frozenset of movieID numbers")

# 对于已经观看的四部电影的用户，可以根据置信度推荐下一部可观看的电影

In [151]:
for freqSet, confiDict in confidenceDict.items():
    for movieID,confidence in confiDict.items():
        if confidence >0.9:
            print('对于观看了{!s}的观众，推荐观看{!s}，置信度为{:.2%}'.format(getTitleByMovieID(freqSet), getTitleByMovieID(movieID), confidence))
            print('--------------------------------------------------------------------------------------------------------------------')

对于观看了Indiana Jones and the Last Crusade (1989),Empire Strikes Back, The (1980),Princess Bride, The (1987),Raiders of the Lost Ark (1981)的观众，推荐观看Star Wars (1977)，置信度为100.00%
--------------------------------------------------------------------------------------------------------------------
对于观看了Star Wars (1977),Terminator, The (1984),Princess Bride, The (1987),Raiders of the Lost Ark (1981)的观众，推荐观看Empire Strikes Back, The (1980)，置信度为91.23%
--------------------------------------------------------------------------------------------------------------------
对于观看了Star Wars (1977),Back to the Future (1985),Forrest Gump (1994),Raiders of the Lost Ark (1981)的观众，推荐观看Empire Strikes Back, The (1980)，置信度为93.64%
--------------------------------------------------------------------------------------------------------------------
对于观看了Indiana Jones and the Last Crusade (1989),Star Wars (1977),Princess Bride, The (1987),Raiders of the Lost Ark (1981)的观众，推荐观看Empire Strikes Back, The (1980)，置信度为94.69%
--

In [128]:
for movieID, confidenceList in confidenceDictEachMovie.items():
    confidenceDictEachMovie[movieID] = np.mean(confidenceList)

In [130]:
from operator import itemgetter

# 找出平均置信度高的电影，这些电影更符合大众口味。可以推荐给新用户。

In [147]:
for movieID,confidence in sorted(confidenceDictEachMovie.items(), key=itemgetter(1), reverse=True)[:5]:
    print('推荐{!s}，平均置信度为{:.2%}'.format(getTitleByMovieID(movieID), confidence))
    print('-----------------------------------------------------------------------')

推荐Star Wars (1977)，平均置信度为97.61%
-----------------------------------------------------------------------
推荐Raiders of the Lost Ark (1981)，平均置信度为93.52%
-----------------------------------------------------------------------
推荐Empire Strikes Back, The (1980)，平均置信度为89.76%
-----------------------------------------------------------------------
推荐Return of the Jedi (1983)，平均置信度为83.42%
-----------------------------------------------------------------------
推荐Silence of the Lambs, The (1991)，平均置信度为75.19%
-----------------------------------------------------------------------
